In [ ]:
input_folder = r"C:\Users\Hieu Pham\Downloads\1"
style = 'một buổi nói chuyện ted talk nghiêm túc'
target_language = 'Japanese'
#Example
#historical drama
#modern drama
#news & current affairs

#The number of subtitle lines to be translated per API call. The code works stably with a value of 1.
#However, the drawback is that it results in a large number of API calls. I’m still working on improving the translation efficiency.
batch_size=1
context_window=100

In [ ]:
#Split the file. The purpose is for translation. You can check how the file is split to understand better. 
#Note that after splitting, the original subtitle file will be deleted to avoid code conflicts when merging. 
#You can either save two versions in separate folders or modify the code so it doesn’t delete the original file.

import os
import re

def split_srt_folder(input_folder):
    for file_name in os.listdir(input_folder):
        if file_name.endswith('.srt'):
            file_path = os.path.join(input_folder, file_name)
            base_name = os.path.splitext(file_name)[0]
            ts_file = os.path.join(input_folder, f"{base_name}_timestamps.txt")
            sub_file = os.path.join(input_folder, f"{base_name}_subtitles.txt")
            
            timestamps = []
            subtitles = []
            
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read().strip()
                blocks = re.split(r'\n\n', content)
                
                for block in blocks:
                    lines = block.split('\n')
                    if len(lines) >= 3:
                        index = lines[0].strip()
                        timestamp = lines[1].strip()
                        subtitle_text = ' '.join(lines[2:]).strip()
                        
                        timestamps.append(f"{index} {timestamp}")
                        subtitles.append(f"{index} {subtitle_text}")
            
            with open(ts_file, 'w', encoding='utf-8') as ts_out:
                ts_out.write('\n'.join(timestamps))
            
            with open(sub_file, 'w', encoding='utf-8') as sub_out:
                sub_out.write('\n'.join(subtitles))
            
            os.remove(file_path)
            print(f"Tách file hoàn tất: {file_name} -> {ts_file}, {sub_file} (Gốc đã xóa)")

split_srt_folder(input_folder)

In [9]:
#main translation code cell
import os
import re
from chat_bot import call_chatbot
from dotenv import load_dotenv

# Load biến môi trường nếu cần
load_dotenv()
gpt_api_key = os.environ.get("CHATGPT_API_KEY")

# Helper: xác định dòng có nội dung để dịch hay không
def is_translatable(line: str) -> bool:
    """Trả về False nếu dòng rỗng hoặc chỉ chứa dấu câu/biểu cảm (không có chữ hoặc số).
    Nếu có ít nhất 1 ký tự chữ (unicode) hoặc chữ số -> True.
    """
    s = line.strip()
    if not s:
        return False
    # Nếu có bất kỳ ký tự chữ hoặc số nào thì coi là có thể dịch
    for ch in s:
        if ch.isalpha() or ch.isdigit():
            return True
    return False


def translate_file(file_path, style, batch_size=1, context_window=100, summary_lines_count=500, target_language='Vietnamese', input_encoding="utf-8", ):
    """
    file_path: đường dẫn file subtitle
    style: phong cách dịch
    batch_size: số dòng gửi 1 lần cho API (1..20). Nếu muốn 1 dòng 1 lần -> batch_size=1
    context_window: tổng số dòng dùng làm context tham khảo (ưu tiên đặt batch ở giữa nếu có thể)
    summary_lines_count: số dòng đầu dùng để tạo summary bối cảnh (mặc định giữ 500 như bạn yêu cầu)
    """
    # clamp batch_size
    if batch_size < 1:
        batch_size = 1
    if batch_size > 20:
        batch_size = 20

    dir_name = os.path.dirname(file_path)
    base_name = os.path.basename(file_path).rsplit('.', 1)[0]
    output_file = os.path.join(dir_name, f"{base_name}_translated.txt")

    with open(file_path, "r", encoding=input_encoding) as f:
        lines = f.readlines()

    # Tóm tắt bối cảnh từ summary_lines_count dòng đầu (bạn chọn giữ 500)
    initial_context_lines = lines[:summary_lines_count]
    initial_context_str = "".join(initial_context_lines)
    prompt_context = f"""
    Dựa trên {summary_lines_count} dòng đầu của file subtitle dưới đây, hãy tóm tắt bối cảnh của nội dung sau đây.

    Dữ liệu:
    {initial_context_str}
    """

    summary_context = call_chatbot(prompt_context, "gpt-5-mini", "chatgpt", gpt_api_key)
    print("--- Summary context ---")
    print(summary_context)
    print("-----------------------")

    with open(output_file, "w", encoding="utf-8") as f_out:
        total = len(lines)
        i = 0
        processed = 0

        while i < total:
            # Lấy một batch (liên tiếp)
            batch_lines = lines[i: i + batch_size]
            batch_indices = list(range(i, min(i + batch_size, total)))

            # Xác định những dòng có thể dịch trong batch
            translatable_items = []  # list of (idx, line)
            for idx, ln in zip(batch_indices, batch_lines):
                if is_translatable(ln):
                    translatable_items.append((idx, ln.rstrip("\n")))

            # Nếu không có dòng nào cần dịch trong batch -> ghi file các dòng tương ứng là rỗng (skipped)
            if not translatable_items:
                for idx in batch_indices:
                    # Ghi index (1-based) và rỗng để giữ mapping
                    f_out.write(f"{idx+1}\t\n")
                    processed += 1
                i += batch_size
                print(f"Đã xử lý {processed}/{total} dòng (batch gồm {len(batch_indices)} dòng, không có nội dung dịch).")
                continue

            # Tính vị trí context để cố gắng đặt batch ở giữa context_window
            # half_before = số dòng trước batch muốn có
            half_before = max(0, (context_window - len(translatable_items)) // 2)
            start = max(0, translatable_items[0][0] - half_before)
            end = min(total, start + context_window)
            # Nếu không đủ context_window (chạm cuối), đẩy start lên
            if end - start < context_window:
                start = max(0, end - context_window)

            context_segment = "".join(lines[start:end])

            # Chuẩn bị prompt: gửi summary + context_segment + các dòng cần dịch rõ ràng
            # Dùng markers để model biết thứ tự và mapping index
            prompt_lines = []
            for idx, ln in translatable_items:
                # Dùng chỉ dẫn rõ ràng, không thêm indexing nội bộ thừa
                prompt_lines.append(f"<<<LINE {idx+1}>>>{ln}")

            prompt_batch = "\n\n".join(prompt_lines)

            prompt = f"""
            Summary (bối cảnh chung):
            {summary_context}

            Context tham khảo (khoảng {context_window} dòng, cố gắng đặt dòng/khối cần dịch ở giữa nếu có thể):
            {context_segment}

            Các dòng sau đây cần dịch sang {target_language} với phong cách "{style}":
            {prompt_batch}

            **Yêu cầu trả về:**
            - Dịch ngắn gọn, ko dài dòng vào thẳng trọng tâm.
            - Chỉ trả về phần **bản dịch** cho mỗi dòng yêu cầu, theo đúng **thứ tự** của các marker <<<LINE N>>>.
            - Mỗi bản dịch **đặt trên 1 dòng mới** (không kèm index, không kèm văn bản gốc, không kèm giải thích).
            - Nếu một dòng trống hoặc chỉ dấu câu thì **không gửi** (chúng tôi đã lọc trước), bạn chỉ cần trả translations cho những dòng thật sự có text.
            - Tránh xưng hô thô tục; giữ lịch sự; dịch ngắn gọn, đúng trọng tâm.
            """

            # Gọi API một lần cho batch
            translated_batch_text = call_chatbot(prompt, "gpt-4o-mini", "chatgpt", gpt_api_key)
            # Tách các dòng trả về
            translated_lines = [ln for ln in translated_batch_text.splitlines() if ln.strip() != ""]

            # Nếu số dòng trả về không khớp số dòng yêu cầu, cố gắng phân tách bằng "\n\n"
            if len(translated_lines) != len(translatable_items):
                alt = [p.strip() for p in re.split(r"\n\s*\n", translated_batch_text) if p.strip() != ""]
                if len(alt) == len(translatable_items):
                    translated_lines = alt

            # Nếu vẫn mismatch: fallback — ghi toàn bộ block vào dòng đầu (an toàn nhưng kém chuẩn)
            if len(translated_lines) != len(translatable_items):
                print("[WARN] Số bản dịch nhận về không khớp với số dòng yêu cầu. Ghi nguyên block trả về cho dòng đầu trong batch.")

            # Ghi kết quả: với các dòng có dịch -> dùng translated_lines; với các dòng skip -> để trống
            t_idx = 0
            for idx in batch_indices:
                if t_idx < len(translatable_items) and idx == translatable_items[t_idx][0]:
                    # có bản dịch tương ứng
                    if len(translated_lines) == len(translatable_items):
                        translation = translated_lines[t_idx].strip()
                    else:
                        # fallback: ghi nguyên block vào bản dịch đầu
                        translation = translated_batch_text.strip() if t_idx == 0 else ""
                    f_out.write(f"{idx+1}\t{translation}\n")
                    t_idx += 1
                else:
                    # dòng trước đó bị coi là không dịch (rỗng/dấu câu) -> giữ rỗng
                    f_out.write(f"{idx+1}\t\n")
                processed += 1

            print(f"Đã xử lý {processed}/{total} dòng... (gọi API cho batch starting at line {i+1})")

            i += batch_size

    print(f"Dịch xong! File đã lưu tại: {output_file}")


def translate_folder(input_folder, style, batch_size=1, context_window=100, summary_lines_count=500, target_language='Vietnamese'):
    if not os.path.exists(input_folder):
        print(f"Thư mục {input_folder} không tồn tại.")
        return

    files = [f for f in os.listdir(input_folder) if f.endswith("subtitles.txt")]
    if not files:
        print("Không tìm thấy file subtitles nào trong thư mục.")
        return

    for file in files:
        file_path = os.path.join(input_folder, file)
        print(f"Bắt đầu dịch file: {file}")
        translate_file(file_path, style, batch_size=batch_size, context_window=context_window, summary_lines_count=summary_lines_count, target_language=target_language)
        print(f"Hoàn thành dịch file: {file}\n")
        
# Giả sử input_folder đã được định nghĩa trước đó
translate_folder(
    input_folder=input_folder, 
    style=style, 
    batch_size=batch_size, 
    context_window=context_window, 
    target_language=target_language
)

Bắt đầu dịch file: A Simple Way to Break a Bad Habit - Judson Brewer - TED_subtitles.txt
--- Summary context ---
Đây là nội dung của một bài nói chuyện (giống phong cách TED talk) về chánh niệm (mindfulness) và cách nó giúp phá vỡ các thói quen có hại. Diễn giả bắt đầu bằng trải nghiệm cá nhân khi học thiền, giải thích vì sao khó duy trì chú ý bằng cách “ép” bản thân, rồi chuyển sang phân tích khoa học về hình thành thói quen (khuôn mẫu: kích hoạt — hành vi — phần thưởng) và cơ chế học tập dựa trên khen thưởng/không khen thưởng. 

Nội dung chính gồm:
- Minh họa bằng ví dụ ăn uống và hút thuốc: cảm xúc hoặc ngữ cảnh làm phát khởi hành vi, dẫn đến hình thành thói quen.
- Giới thiệu phương pháp chánh niệm dựa trên “tò mò” (không cưỡng chế), ví dụ bài tập “hút thuốc một cách chánh niệm” khiến người tham gia nhận thấy mùi/vị khó chịu và mất hứng.
- Lý giải thần kinh: khi bị cuốn vào cơn thèm thì mạng lưới mặc định (default mode network), đặc biệt vùng posterior cingulate, hoạt động; chánh n

In [ ]:
#code to merge the timestamp file and the translated file into a complete SRT subtitle file
                
def merge_srt_folder(input_folder):
    for file_name in os.listdir(input_folder):
        if file_name.endswith('_timestamps.txt'):
            base_name = file_name.replace('_timestamps.txt', '')
            ts_file = os.path.join(input_folder, file_name)
            sub_file = os.path.join(input_folder, f"{base_name}_subtitles_translated.txt")
            output_file = os.path.join(input_folder, f"{base_name}_merged.srt")

            if os.path.exists(sub_file):
                timestamps = {}
                subtitles = {}

                # Đọc file timestamps
                with open(ts_file, 'r', encoding='utf-8') as ts_in:
                    for line in ts_in:
                        parts = line.strip().split(maxsplit=1)  # Tách tối đa 1 lần
                        if len(parts) == 2:
                            timestamps[parts[0]] = parts[1]

                # Đọc file subtitles
                with open(sub_file, 'r', encoding='utf-8') as sub_in:
                    for line in sub_in:
                        parts = line.strip().split(maxsplit=1)  # Tách tối đa 1 lần
                        if len(parts) == 2:
                            subtitles[parts[0]] = parts[1]

                # Ghi file .srt
                with open(output_file, 'w', encoding='utf-8') as out:
                    for index in sorted(timestamps.keys(), key=int):
                        timestamp = timestamps[index]
                        subtitle = subtitles.get(index, " ")  # Nếu không có sub, để khoảng trắng
                        out.write(f"{index}\n{timestamp}\n{subtitle}\n\n")

                print(f"Gộp file hoàn tất: {output_file}")



merge_srt_folder(input_folder)

The following code snippets are intended for fine-tuning when converting from SRT to an audio file. You can check the corresponding project in another repo of mine.
Use with caution — testing is required first.
Merging timestamps means combining adjacent timestamps that are too close together, in order to prevent the voice from being cut off due to subtitles being too short.

In [ ]:
import os
import re
from datetime import timedelta

def parse_time(time_str):
    """Chuyển đổi chuỗi thời gian SRT sang đối tượng timedelta."""
    parts = re.match(r'(\d{2}):(\d{2}):(\d{2}),(\d{3})', time_str.strip())
    if not parts:
        raise ValueError(f"Định dạng thời gian không hợp lệ: {time_str}")
    hours, minutes, seconds, milliseconds = map(int, parts.groups())
    return timedelta(hours=hours, minutes=minutes, seconds=seconds, milliseconds=milliseconds)

def format_time(td):
    """Chuyển đổi đối tượng timedelta sang chuỗi thời gian SRT."""
    if td.total_seconds() < 0:
         return f"00:00:00,000" # Tránh thời gian âm trong SRT

    total_seconds = td.total_seconds()
    hours = int(total_seconds // 3600)
    minutes = int((total_seconds % 3600) // 60)
    seconds = int(total_seconds % 60)
    milliseconds = int(td.microseconds // 1000)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d},{milliseconds:03d}"

def process_srt_file(file_path, gap_ms=100):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file {file_path}")
        return
    except Exception as e:
        print(f"Lỗi khi đọc file {file_path}: {e}")
        return

    subs = []
    raw_blocks = re.split(r'\r?\n\s*\r?\n', content.strip())
    seq_counter = 1

    for block_num, block_content in enumerate(raw_blocks):
        if not block_content.strip():
            continue

        block_pattern = re.match(
            r'(\d+)\s*\r?\n'
            r'(\d{2}:\d{2}:\d{2},\d{3})\s*-->\s*(\d{2}:\d{2}:\d{2},\d{3})\s*\r?\n?'
            r'(.*)',
            block_content,
            re.DOTALL
        )

        if block_pattern:
            original_seq = block_pattern.group(1)
            start_time_str = block_pattern.group(2)
            end_time_str = block_pattern.group(3)
            text = block_pattern.group(4).strip() # .strip() quan trọng ở đây

            try:
                start_time = parse_time(start_time_str)
                end_time = parse_time(end_time_str)
                if end_time < start_time:
                    # print(f"Cảnh báo file {file_path}, sub gốc {original_seq}: end_time < start_time. Sửa end_time = start_time.")
                    end_time = start_time
                
                subs.append({
                    'seq': str(seq_counter),
                    'start_time': start_time,
                    'end_time': end_time,
                    'text': text, # text đã được .strip()
                    'original_start_str': start_time_str,
                })
                seq_counter += 1
            except ValueError as e:
                print(f"Lỗi khi xử lý thời gian trong file {file_path}, sub gốc {original_seq} (khối {block_num+1}): {e}")
                continue
        else:
            print(f"Cảnh báo: Không thể parse khối sub trong {file_path} (khối {block_num+1}): \"{block_content[:100]}...\"")

    if not subs:
        print(f"Không tìm thấy sub nào hợp lệ trong file {file_path} hoặc định dạng không đúng.")
        return

    # Điều chỉnh end_time
    for i in range(len(subs) - 1):
        current_sub = subs[i]
        next_sub = subs[i+1]
        target_end_time = next_sub['start_time'] - timedelta(milliseconds=gap_ms)
        if target_end_time > current_sub['start_time']:
            current_sub['end_time'] = target_end_time
        else:
            current_sub['end_time'] = current_sub['start_time'] + timedelta(milliseconds=1)
    
    if subs:
        last_sub = subs[-1]
        if last_sub['end_time'] < last_sub['start_time']:
            last_sub['end_time'] = last_sub['start_time'] + timedelta(milliseconds=1)

    # Ghi đè file cũ với logic dòng trống đã sửa
    try:
        with open(file_path, 'w', encoding='utf-8') as f:
            srt_output_lines = []
            num_subs = len(subs)
            for i, sub_item in enumerate(subs):
                srt_output_lines.append(f"{sub_item['seq']}")
                srt_output_lines.append(f"{sub_item['original_start_str']} --> {format_time(sub_item['end_time'])}")
                
                # Thêm dòng text (có thể là chuỗi rỗng "", sẽ được join thành một dòng trống)
                # sub_item['text'] đã được strip() trong quá trình parsing.
                # Nếu text có nhiều dòng, các \n bên trong sẽ được giữ nguyên.
                srt_output_lines.append(sub_item['text'])
                
                # Thêm dòng trống separator nếu không phải sub cuối cùng
                if i < num_subs - 1:
                    # Luôn thêm một dòng trống làm separator chính.
                    # Nếu sub_item['text'] là rỗng (""), dòng srt_output_lines.append(sub_item['text']) ở trên
                    # đã tạo ra dòng trống thứ nhất (cho nội dung). Dòng trống này là dòng thứ hai (separator).
                    # Nếu sub_item['text'] có nội dung, đây sẽ là dòng trống duy nhất sau text.
                    srt_output_lines.append("") 
            
            final_srt_string = "\n".join(srt_output_lines)
            
            # Đảm bảo file kết thúc bằng một ký tự newline duy nhất (nếu file không rỗng)
            if final_srt_string:
                final_srt_string = final_srt_string.rstrip('\r\n') + '\n'
            
            f.write(final_srt_string)
        print(f"Đã xử lý và ghi đè file: {file_path}")
    except Exception as e:
        print(f"Lỗi khi ghi file {file_path}: {e}")

def process_srt_folder(folder_path, gap_ms=100):
    if not os.path.isdir(folder_path):
        print(f"Lỗi: Thư mục '{folder_path}' không tồn tại.")
        return
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(".srt"):
            file_path = os.path.join(folder_path, filename)
            print(f"Đang xử lý file: {file_path}")
            process_srt_file(file_path, gap_ms)
    print("Hoàn tất xử lý thư mục.")


desired_gap_ms = 100
process_srt_folder(input_folder, desired_gap_ms)

This code cell may not be very useful — use it carefully. Especially if you’ve already manually cleaned the subtitles thoroughly and shortened subtitle lines that are unnecessarily long, you probably don’t need to use the code below.

It merges subtitles, meaning it combines adjacent subtitle entries. For example, if subtitle A’s length exceeds its expected time window and subtitle B immediately after A has a duration longer than its actual content, the two subtitles will be merged to prevent subtitle A from being cut off. Audio quality may degrade significantly. The merge criteria are handled by a function that takes the standard speaking rate as an input — see the code for details.

In [ ]:
import os
import re
from datetime import timedelta

# --- Constants ---
STANDARD_READING_SPEED = 20  # Kí tự / giây 

# --- Helper Functions ---

def srt_time_to_milliseconds(time_str):
    """Chuyển đổi chuỗi thời gian SRT (HH:MM:SS,ms) sang milliseconds."""
    match = re.match(r'(\d{2}):(\d{2}):(\d{2}),(\d{3})', time_str)
    if not match:
        raise ValueError(f"Định dạng thời gian không hợp lệ: {time_str}")
    h, m, s, ms = map(int, match.groups())
    return (h * 3600 + m * 60 + s) * 1000 + ms

def milliseconds_to_srt_time(ms_total):
    """Chuyển đổi milliseconds sang chuỗi thời gian SRT (HH:MM:SS,ms)."""
    if ms_total < 0:
        ms_total = 0 # Đảm bảo không có thời gian âm
    seconds_total = ms_total // 1000
    ms = ms_total % 1000
    m, s = divmod(seconds_total, 60)
    h, m = divmod(m, 60)
    return f"{h:02d}:{m:02d}:{s:02d},{ms:03d}"

def calculate_reading_speed(text, start_ms, end_ms):
    """Tính tốc độ đọc (kí tự / giây)."""
    char_count = len(text)
    duration_ms = end_ms - start_ms
    if duration_ms <= 0: # Tránh chia cho 0 hoặc thời gian âm
        return float('inf') # Tốc độ vô hạn nếu không có thời gian hiển thị
    duration_seconds = duration_ms / 1000.0
    return char_count / duration_seconds

class SubtitleItem:
    """Lớp đại diện cho một khối subtitle."""
    def __init__(self, index, start_time_str, end_time_str, text_lines):
        self.index = int(index)
        self.start_time_str = start_time_str
        self.end_time_str = end_time_str
        self.text = "\n".join(text_lines).strip() # Nối các dòng text và loại bỏ khoảng trắng thừa

        self.start_ms = srt_time_to_milliseconds(start_time_str)
        self.end_ms = srt_time_to_milliseconds(end_time_str)

    def __str__(self):
        """Biểu diễn chuỗi cho việc ghi file SRT."""
        return f"{self.index}\n{self.start_time_str} --> {self.end_time_str}\n{self.text}\n"

    def update_times_and_text(self, new_start_ms, new_end_ms, new_text):
        """Cập nhật thời gian và nội dung cho subtitle (sau khi gộp)."""
        self.start_ms = new_start_ms
        self.end_ms = new_end_ms
        self.text = new_text.strip()
        self.start_time_str = milliseconds_to_srt_time(new_start_ms)
        self.end_time_str = milliseconds_to_srt_time(new_end_ms)

    def get_reading_speed(self):
        return calculate_reading_speed(self.text, self.start_ms, self.end_ms)

def parse_srt_file(filepath):
    """Đọc và phân tích nội dung file SRT."""
    subtitles = []
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            content = f.read().strip()
    except Exception as e:
        print(f"Lỗi khi đọc file {filepath}: {e}")
        return []

    if not content:
        return []

    # Regex để match một khối subtitle hoàn chỉnh
    # Nó sẽ match index, time_range, và text (kể cả text nhiều dòng)
    # (?s) là cờ DOTALL, cho phép . khớp với cả ký tự xuống dòng
    # (?:^\d+\s*$) là non-capturing group để tìm dòng trống hoặc dòng chỉ chứa số (chỉ số của sub tiếp theo)
    # làm dấu hiệu kết thúc của text block hiện tại.
    # Tuy nhiên, cách đơn giản hơn là split bằng \n\n
    
    blocks = content.split('\n\n')
    
    for block_str in blocks:
        if not block_str.strip():
            continue
        lines = block_str.strip().split('\n')
        if len(lines) < 3:
            # print(f"Cảnh báo: Khối không hợp lệ trong {filepath}:\n{block_str}")
            continue # Bỏ qua khối không hợp lệ

        index = lines[0]
        time_line = lines[1]
        text_lines = lines[2:]

        time_match = re.match(r'(\d{2}:\d{2}:\d{2},\d{3})\s*-->\s*(\d{2}:\d{2}:\d{2},\d{3})', time_line)
        if not time_match:
            # print(f"Cảnh báo: Dòng thời gian không hợp lệ trong {filepath}: {time_line}")
            continue

        start_time_str, end_time_str = time_match.groups()
        
        try:
            subtitles.append(SubtitleItem(index, start_time_str, end_time_str, text_lines))
        except ValueError as e:
            print(f"Lỗi khi xử lý thời gian trong {filepath} cho khối: {block_str}\nLỗi: {e}")
            continue
            
    return subtitles

def process_srt_data(subtitles):
    """Xử lý logic gộp subtitle."""
    if not subtitles:
        return []

    processed_subs = []
    i = 0
    while i < len(subtitles):
        current_sub = subtitles[i]
        
        # Khởi tạo subtitle sẽ được gộp (ban đầu là chính nó)
        merged_start_ms = current_sub.start_ms
        merged_end_ms = current_sub.end_ms
        merged_text_parts = [current_sub.text]
        
        # Chỉ số của sub tiếp theo sẽ được gộp (nếu cần)
        # current_sub_index_in_original_list là chỉ số của current_sub trong danh sách subtitles ban đầu
        # next_sub_to_merge_index là chỉ số của sub tiếp theo *trong danh sách subtitles ban đầu*
        next_sub_to_merge_index = i + 1

        while True:
            # Tính tốc độ đọc cho khối đã gộp (hoặc chưa gộp)
            current_speed = calculate_reading_speed(".".join(merged_text_parts), merged_start_ms, merged_end_ms)

            if current_speed <= STANDARD_READING_SPEED:
                # Tốc độ đọc ổn, không cần gộp thêm nữa với sub này
                break
            
            # Tốc độ đọc quá cao, cần gộp với sub tiếp theo
            if next_sub_to_merge_index >= len(subtitles):
                # Không còn sub nào để gộp
                break
            
            # Lấy sub tiếp theo để gộp
            sub_to_add = subtitles[next_sub_to_merge_index]
            
            # Cập nhật thông tin cho khối gộp
            merged_text_parts.append(sub_to_add.text)
            merged_end_ms = sub_to_add.end_ms # Quan trọng: end_ms của sub cuối cùng trong khối gộp
            
            next_sub_to_merge_index += 1 # Chuyển sang sub tiếp theo nữa nếu vòng lặp này tiếp tục

        # Tạo một SubtitleItem mới cho khối đã gộp (hoặc không gộp)
        # Index sẽ được cập nhật lại sau
        final_merged_text = ". ".join(merged_text_parts) # Nối các phần text bằng ". "
        
        # Tạo subtitle mới từ các phần đã gộp
        # Index sẽ được gán lại sau
        new_sub = SubtitleItem(
            index=0, # Sẽ được cập nhật lại index
            start_time_str=milliseconds_to_srt_time(merged_start_ms),
            end_time_str=milliseconds_to_srt_time(merged_end_ms),
            text_lines=[final_merged_text] # text_lines là một list
        )
        processed_subs.append(new_sub)
        
        # Di chuyển con trỏ i đến sau sub cuối cùng đã được gộp vào khối hiện tại
        i = next_sub_to_merge_index
        
    # Cập nhật lại index cho các subtitle đã xử lý
    for idx, sub_item in enumerate(processed_subs):
        sub_item.index = idx + 1
        
    return processed_subs

def write_srt_file(filepath, subtitles):
    """Ghi danh sách subtitle đã xử lý vào file, ghi đè file gốc."""
    try:
        with open(filepath, 'w', encoding='utf-8') as f:
            for sub_item in subtitles:
                f.write(str(sub_item)) # Sử dụng __str__ của SubtitleItem
                f.write('\n') # Thêm dòng trống giữa các khối sub
        # print(f"Đã xử lý và ghi đè file: {filepath}")
    except Exception as e:
        print(f"Lỗi khi ghi file {filepath}: {e}")

def process_folder(folder_path):
    """Xử lý tất cả các file .srt trong một thư mục."""
    found_srt_files = False
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(".srt"):
            found_srt_files = True
            filepath = os.path.join(folder_path, filename)
            print(f"Đang xử lý file: {filepath}...")
            
            original_subtitles = parse_srt_file(filepath)
            if not original_subtitles:
                print(f"Bỏ qua file rỗng hoặc không thể phân tích: {filepath}")
                continue
                
            processed_subtitles = process_srt_data(original_subtitles)
            
            if processed_subtitles:
                write_srt_file(filepath, processed_subtitles)
                print(f"Đã xử lý và ghi đè thành công: {filepath}")
            else:
                print(f"Không có gì để ghi cho file: {filepath} (có thể do lỗi phân tích)")

    if not found_srt_files:
        print(f"Không tìm thấy file .srt nào trong thư mục: {folder_path}")



process_folder(input_folder)